In [28]:
import pandas as pd
import os
from tqdm import tqdm
from dateutil.relativedelta import *
from datetime import datetime as dt

date_index_reference = 'issue_sheet_index.csv'
# chunksize = 1000
date_format = '%d-%b-%y'
custom_date_parser = lambda x: dt.strptime(x, date_format)

if not os.path.isfile(date_index_reference):
    print('date index needs to be instantiated, proceeding to do that')
    #Initialize the index df
    index_df = pd.DataFrame([], columns=['date_issue', 'sheet_path', 'sheet_num'])

    formatted_csv_path = '../00_The_Economist_Scraper/formatted_csvs/'
    formatted_csv_stub = "formatted_sub_article_text"
    suffix = '.csv'

    # GOING THROUGH EACH FORMATTED CSV
    for sub_csv_index in tqdm(range(0,124)):
        read_path = formatted_csv_path + formatted_csv_stub + str(sub_csv_index) + suffix
        temp_df =  pd.read_csv(read_path, parse_dates=['date'], date_parser=custom_date_parser)

        # Identifying unique dates
        temp_df = temp_df.drop_duplicates(subset=['date'], keep='first')
        temp_df = temp_df['date']

        temp_index = pd.DataFrame([], columns=['date_issue', 'sheet_path', 'sheet_num'])

        # Pulling out the info of interest 
        for issue_row in temp_df.index.tolist():
            row_obs = temp_df.loc[issue_row]
            # recall that temp_df is now just a single column of 'date
            temp_index.loc[issue_row, 'date_issue'] = temp_df.loc[issue_row].date()
            temp_index.loc[issue_row, 'sheet_path'] = formatted_csv_path + formatted_csv_stub + str(sub_csv_index) + suffix
            temp_index.loc[issue_row, 'sheet_num'] = sub_csv_index
        ### REMEMBER THAT A DATE MIGHT APPEAR IN/CROSS OVER MORE THAN ONE SHEET
        # Need to concat and reset indices because first instances of different issues may appear on the same row (i.e. "index" of temp_df) of different sheets. Need to do this to avoid conflicts
        index_df = pd.concat([index_df, temp_index], ignore_index=True)
        index_df.reset_index(drop=True, inplace=True)
        index_df['date_issue'] = pd.to_datetime(index_df['date_issue'])

    index_df.to_csv(date_index_reference, index=False)
        
else:
    print('Issues are already indexed for their dates, carry on')
    index_df = pd.read_csv(date_index_reference)

print(index_df.head(5))


date index needs to be instantiated, proceeding to do that


100%|██████████| 124/124 [00:11<00:00, 10.88it/s]

  date_issue                                         sheet_path sheet_num
0 1992-01-04  ../00_The_Economist_Scraper/formatted_csvs/for...         0
1 1992-01-11  ../00_The_Economist_Scraper/formatted_csvs/for...         0
2 1992-02-15  ../00_The_Economist_Scraper/formatted_csvs/for...         0
3 1992-02-22  ../00_The_Economist_Scraper/formatted_csvs/for...         0
4 1992-02-29  ../00_The_Economist_Scraper/formatted_csvs/for...         0


In [29]:
### READING IN LEADER TERM DATA
leader_term_name_df = pd.read_csv('all_leaders_econ_styling.csv', encoding='latin1')
leader_term_name_df = leader_term_name_df.drop_duplicates(subset='leadid')


leader_term_name_df['term_start'] = dt.now
leader_term_name_df['term_end'] =dt.now

# leader_term_name_df
for index in tqdm(leader_term_name_df.index.tolist()):
    # FORMAT: start = dt(year, month, day)
    leader_term_name_df.loc[index, 'term_start'] = dt(leader_term_name_df.loc[index, 'start_year'], leader_term_name_df.loc[index, 'start_month'], leader_term_name_df.loc[index, 'start_date'])
    
    # Push to the end of the month as far as possible (accounting for leap years)
    # Note that we are pushing the end of the term out 6 months anyway (in case we miss an issue printed on the 29th-31st)
    leader_term_name_df.loc[index, 'term_end'] = dt(leader_term_name_df.loc[index, 'end_year'], leader_term_name_df.loc[index, 'end_month'], 28)

print(leader_term_name_df.head(5))


100%|██████████| 560/560 [00:00<00:00, 2082.30it/s]

    ccode  country  year     leadid  polity  start_date  start_month  \
0     339  Albania  2002  A2.9-4924       7          29           10   
1     339  Albania  2002  A2.9-4927       7          22            2   
2     339  Albania  2002  A2.9-4930       7          31            7   
6     339  Albania  2005    A3.0-79       9          11            9   
15    339  Albania  2013   cb-339-1       9          15            9   

    start_year  end_month  end_year  gender econ_style_first econ_style_last  \
0         1999          2      2002       1             ilir            meta   
1         2002          7      2002       1          pandeli           majko   
2         2002          9      2005       1            fatos            nano   
6         2005          9      2013       1             sali         berisha   
15        2013          8      2021       1              edi            rama   

   econ_style_alias      leader           term_start             term_end  
0         

In [31]:
# MATCHING TERM-WINDOWS (term length +/- 6 months) WITH ECONOMIST ISSUE DATES
# We then pair this with the dat_index_reference.csv to figure out which chunks of formatted_compiled_articles.csv we should read
six_month_margin = relativedelta(months = 6)

leaders_windows_indices_df = leader_term_name_df

# Constructing the 6 month window around the term start
leaders_windows_indices_df = leaders_windows_indices_df.assign(leader_aprox_starts = lambda df: df.term_start - six_month_margin)
leaders_windows_indices_df = leaders_windows_indices_df.assign(leader_aprox_ends = lambda df: df.term_end + six_month_margin)


# Merge window-start and window-end dates with the "nearest" (FORWARD OR BACKWARD) Economist issue date.
# Read in that issue's starting row/index
leaders_windows_indices_df = leaders_windows_indices_df.sort_values(by=['leader_aprox_starts'])
leaders_windows_indices_df['leader_aprox_starts'] = pd.to_datetime(leaders_windows_indices_df['leader_aprox_starts'])


leaders_windows_indices_df = pd.merge_asof(left=leaders_windows_indices_df, right=index_df, left_on='leader_aprox_starts', right_on='date_issue', direction='nearest')
leaders_windows_indices_df = leaders_windows_indices_df.rename(columns={'date_issue' : 'date_start_issue', 'sheet_path' : 'start_sheet_path', 'sheet_num' : 'start_sheet_num'})

################################################
# Merge window-start and window-end dates with the "nearest" (FORWARD OR BACKWARD) Economist issue date.
# Read in that issue's ending row/index
################################################
leaders_windows_indices_df = leaders_windows_indices_df.sort_values(by=['leader_aprox_ends'])
leaders_windows_indices_df = pd.merge_asof(left=leaders_windows_indices_df, right=index_df, left_on='leader_aprox_ends', right_on='date_issue', direction='nearest')


leaders_windows_indices_df = leaders_windows_indices_df.rename(columns={'date_issue': 'date_end_issue', 'sheet_path': 'end_sheet_path', 'sheet_num' : 'end_sheet_num'})



################################################
# Generating number of rows that need to be read-in from the formatted_compiled_articles.csv file
################################################
# leaders_windows_indices_df = leaders_windows_indices_df.assign(nrows = lambda df: df.issue_ending_row - df.issue_starting_row - 1)


################################################
# Add in titles and adjectives
################################################
adjectives_df = pd.read_csv('national_titles_adjectives.csv').drop(columns=['country'])
leaders_windows_indices_df = pd.merge(leaders_windows_indices_df, adjectives_df, how='left', on='ccode')


print(len(leader_term_name_df))

560
